In [1]:
import os
import pandas as pd
from turkish_string import title_tr

# Data Pre-Processing
We will read, pre-process and save excel files.<br>
First let's define some dictionaries to translate column names.

In [2]:
# Common function to pre-process region-base files
def process_column(col, col_dicts):
    parts = col.split(" ve ")
    # Extract multi-level columns and translete using related dictionaries
    result = []
    for i, col_dict in enumerate(col_dicts):
          result.append(col_dict.get(parts[i], parts[i] ))
    return tuple(result)

def data_preprocess(file_path,geo_scale,col_dicts,names,need_pivot_conversion=False):
    skiprows=5 # skip 5 rows for countrywide files
    new_col_names= {'Unnamed: 0': 'year'}
    new_index = ["year"]
    place_colum_dict={"ibbs3":"province","ibbs2":"sub-region","ibbs1":"region","district":"district"}
    if "turkey" not in geo_scale:
        new_col_names["Unnamed: 1"] =  "place"
        new_index.append(place_colum_dict[geo_scale])
        skiprows=4 # skip 4 rows for regional scale files

    df= pd.read_excel(file_path,skiprows=skiprows)

    df.rename(columns= new_col_names, inplace=True)
    df["year"]=df["year"].ffill()#fillna(method='ffill')
    df["year"] = df["year"].astype(int)

    if geo_scale != "district":# remove province codes at the end
        df[place_colum_dict[geo_scale]] = df["place"].str.split('-').str[0]
    elif geo_scale=="district": # extract province and district names from Place column, drop Place column
        df["province"] = df["place"] .str.extract(r'(\w+)\(')
        df["district"] = df["place"] .str.extract(r'\((.*?)\)')

        new_index=["year","province","district"]
    df.drop(columns=["place"],inplace=True)

    df.set_index(new_index, inplace=True)
    df = df.fillna(0)

    df.sort_index(inplace=True)
    if need_pivot_conversion:
        df=pd.pivot_table(df, values=df.columns[1], index=df.index, columns=[df.columns[0]], fill_value=0)
    df=df.astype(int)
    if len(col_dicts)>1:
        # Convert to MultiIndex
        # Apply the helper function to each column name
        tuples = [process_column(col, col_dicts) for col in df.columns]
        # Create the MultiIndex
        multi_index = pd.MultiIndex.from_tuples(tuples, names=names)
        df.columns = multi_index
    else: # not multiindex (col_dicts contains single dictionary)
        df.rename(columns=col_dicts[0], inplace=True)
     
    return df       

### 1.Population Age-Sex
### 1.1. Population Age-Sex Provinces (ibbs3)

In [3]:
dict_age = {"Bilinmeyen":"unknown"}
dict_sex = {"Erkek":"male","Kadın":"female"}

In [124]:
# We change only geo_scale as ibbs3
file_path ="data/raw/population/population-age-sex-ibbs3-2007-2023.xlsx"
df_provinces = data_preprocess(file_path,col_dicts=[dict_sex,dict_age],geo_scale="ibbs3",names=["sex","age_group"])
df_provinces.to_csv("data/preprocessed/population/age-sex-ibbs3-2007-2023.csv")
df_provinces.head(2)

sex             male                                                           \
age_group        0-4   10-14  15-19  20-24  25-29  30-34  35-39  40-44  45-49   
year province                                                                   
2007 Adana     87875  100851  96648  78022  90764  80670  70980  68068  58558   
     Adıyaman  31114   35241  32872  24563  25786  20867  17573  15887  11527   

sex                   ... female                                            \
age_group      50-54  ...  50-54  55-59    5-9  60-64  65-69  70-74  75-79   
year province         ...                                                    
2007 Adana     52112  ...  51402  39610  93870  27931  22227  16919  13775   
     Adıyaman  10253  ...  10555   9782  32072   7440   6242   4572   3973   

sex                              
age_group     80-84 85-89   90+  
year province                    
2007 Adana     7869  2810  1761  
     Adıyaman  2497  1103   656  

[2 rows x 38 columns]

In [125]:
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_provinces = pd.read_csv("data/preprocessed/population/age-sex-ibbs3-2007-2023.csv",index_col=[0,1],header=[0,1])
df_provinces.head(2)

sex             male                                                           \
age_group        0-4   10-14  15-19  20-24  25-29  30-34  35-39  40-44  45-49   
year province                                                                   
2007 Adana     87875  100851  96648  78022  90764  80670  70980  68068  58558   
     Adıyaman  31114   35241  32872  24563  25786  20867  17573  15887  11527   

sex                   ... female                                            \
age_group      50-54  ...  50-54  55-59    5-9  60-64  65-69  70-74  75-79   
year province         ...                                                    
2007 Adana     52112  ...  51402  39610  93870  27931  22227  16919  13775   
     Adıyaman  10253  ...  10555   9782  32072   7440   6242   4572   3973   

sex                              
age_group     80-84 85-89   90+  
year province                    
2007 Adana     7869  2810  1761  
     Adıyaman  2497  1103   656  

[2 rows x 38 columns]

Number of districts has changed from 923 to 973. <br>Since districts data is available since 2018 for 973 districts, we deal with it seperately.

### 1.2. Population Age-Sex district

In [126]:
# Unlike the first three datasets (2007-2023 period), this dataset only includes data between 2018 and 2023.
file_path = "data/raw/population/population-age-sex-district-2018-2023.xlsx"
df_districts= data_preprocess(file_path, geo_scale="district",col_dicts=[dict_sex,dict_age],names=["sex","age_group"],need_pivot_conversion=False)
df_districts.to_csv("data/preprocessed/population/age-sex-district-2018-2023.csv")
df_districts.head(2)    

sex                     male                                                  \
age_group                0-4 10-14 15-19 20-24 25-29 30-34 35-39 40-44 45-49   
year province district                                                         
2018 Adana    Aladağ     641   627   562   622   539   644   712   544   550   
              Ceyhan    6881  6963  6963  5951  5820  5836  5955  5053  4741   

sex                           ... female                                      \
age_group              50-54  ...  50-54 55-59   5-9 60-64 65-69 70-74 75-79   
year province district        ...                                              
2018 Adana    Aladağ     459  ...    448   530   628   501   350   332   229   
              Ceyhan    4784  ...   4710  4577  6580  3778  2848  2007  1483   

sex                                      
age_group              80-84 85-89  90+  
year province district                   
2018 Adana    Aladağ     187    88   36  
              Ceyhan    1025   633  268  

[2 rows x 38 columns]

In [127]:
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_provinces = pd.read_csv("data/preprocessed/population/age-sex-district-2018-2023.csv",index_col=[0,1,2],header=[0,1])
df_provinces.head(2)

sex                     male                                                  \
age_group                0-4 10-14 15-19 20-24 25-29 30-34 35-39 40-44 45-49   
year province district                                                         
2018 Adana    Aladağ     641   627   562   622   539   644   712   544   550   
              Ceyhan    6881  6963  6963  5951  5820  5836  5955  5053  4741   

sex                           ... female                                      \
age_group              50-54  ...  50-54 55-59   5-9 60-64 65-69 70-74 75-79   
year province district        ...                                              
2018 Adana    Aladağ     459  ...    448   530   628   501   350   332   229   
              Ceyhan    4784  ...   4710  4577  6580  3778  2848  2007  1483   

sex                                      
age_group              80-84 85-89  90+  
year province district                   
2018 Adana    Aladağ     187    88   36  
              Ceyhan    1025   633  268  

[2 rows x 38 columns]

## 2.  Population Age-Sex-Maritial Status (Over age 15)
### 2.1. Population Age-Sex-Maritial Status Provinces (ibbs3)

Since 50000 rows are allowed for queries in Tuik Portal, we deal with 4 files. <br>
After preprocessing we will merge them.

In [128]:
file_folder = "data/raw/population/age-sex-maritial-status"
dict_maritial_status={"Bilinmeyen":"unknown","Boşandı":"divorced","Eşi Öldü":"widowed","Hiç Evlenmedi":"never_married","Evli":"married",}
names = ["sex","age_group","maritial_status"]
geo_scale="ibbs3"
df = pd.DataFrame()
for file_name in os.listdir(file_folder):
    if "ibbs3" in file_name:
        file_path = os.path.join(file_folder,file_name)
        # merge 4 dataframes (containing 4 year periods)
        df = pd.concat( [df,data_preprocess(file_path,geo_scale, [dict_sex,dict_age,dict_maritial_status],names,need_pivot_conversion=False)] )
        
# save the merged data frame 
df.to_csv("data/preprocessed/population/age-sex-maritial-status-ibbs3-2008-2023.csv")
df.head(2)

sex                male                                                 \
age_group         15-19                                          20-24   
maritial_status unknown divorced widowed married never_married unknown   
year province                                                            
2008 Adana          0.0        2       2     420         95993     0.0   
     Adıyaman       0.0        2       0     122         32770     0.0   

sex                                                     ...  female           \
age_group                                               ...   85-89            
maritial_status divorced widowed married never_married  ... unknown divorced   
year province                                           ...                    
2008 Adana           108       8    9225         71645  ...     0.0       54   
     Adıyaman         14       0    3439         23108  ...     0.0       13   

sex                                                                     \
age_group                                         90+                    
maritial_status widowed married never_married unknown divorced widowed   
year province                                                            
2008 Adana         1970     263           107     0.0       27     859   
     Adıyaman       679     164            68     0.0        7     346   

sex                                    
age_group                              
maritial_status married never_married  
year province                          
2008 Adana           73            43  
     Adıyaman        54            32  

[2 rows x 151 columns]

We can read from file as below. Note that df has three levels in columns(header=[0,1,2]).

In [129]:
pd.read_csv("data/preprocessed/population/age-sex-maritial-status-ibbs3-2008-2023.csv", index_col=[0, 1], header=[0, 1, 2]).head(2)

sex                male                                                 \
age_group         15-19                                          20-24   
maritial_status unknown divorced widowed married never_married unknown   
year province                                                            
2008 Adana          0.0        2       2     420         95993     0.0   
     Adıyaman       0.0        2       0     122         32770     0.0   

sex                                                     ...  female           \
age_group                                               ...   85-89            
maritial_status divorced widowed married never_married  ... unknown divorced   
year province                                           ...                    
2008 Adana           108       8    9225         71645  ...     0.0       54   
     Adıyaman         14       0    3439         23108  ...     0.0       13   

sex                                                                     \
age_group                                         90+                    
maritial_status widowed married never_married unknown divorced widowed   
year province                                                            
2008 Adana         1970     263           107     0.0       27     859   
     Adıyaman       679     164            68     0.0        7     346   

sex                                    
age_group                              
maritial_status married never_married  
year province                          
2008 Adana           73            43  
     Adıyaman        54            32  

[2 rows x 151 columns]

### 2.2. Population Age-Sex-Maritial Status District

In [23]:
df = pd.DataFrame()
file_folder = "data/raw/population/age-sex-maritial-status"
geo_scale = "district"
dict_sex = {"Erkek":"male","Kadın":"female"}
for file_name in os.listdir(file_folder):
    if "district" in file_name:
        file_path = os.path.join(file_folder,file_name)
        # merge 12 dataframes (containing 1-year periods, each year containing 3 groups of provinces)
        df = pd.concat( [df,data_preprocess(file_path,geo_scale, [dict_sex,dict_age,dict_maritial_status],names,need_pivot_conversion=False)] )
        
# save the merged data frame 
df.to_csv("data/preprocessed/population/age-sex-maritial-status-district-2018-2023.csv")
df.head(2)

NameError: name 'dict_maritial_status' is not defined

In [152]:
pd.read_csv("data/preprocessed/population/age-sex-maritial-status-district-2018-2023.csv",index_col=[0,1,2],header=[0,1,2]).head(3)

sex                        male                                         \
age_group                 15-19                          20-24           
maritial_status        divorced married never_married divorced widowed   
year province district                                                   
2018 Adana    Aladağ          0       3           559        1       0   
              Ceyhan          0      12          6946       20       0   
              Feke            0       1           725        2       0   

sex                                                                    ...  \
age_group                                       25-29                  ...   
maritial_status        married never_married divorced widowed married  ...   
year province district                                                 ...   
2018 Adana    Aladağ        35           586        4       0     164  ...   
              Ceyhan       566          5361       79       2    2496  ...   
              Feke          42           582        7       1     148  ...   

sex                           female                                         \
age_group                      80-84    85-89                                 
maritial_status        never_married divorced widowed married never_married   
year province district                                                        
2018 Adana    Aladağ               5        0      67      16             5   
              Ceyhan              37        9     529      70            24   
              Feke                 4        1      80      21             1   

sex                                                              male  
age_group                   90+                                 15-19  
maritial_status        divorced widowed married never_married widowed  
year province district                                                 
2018 Adana    Aladağ          1      31       2             2     NaN  
              Ceyhan          4     237      22             5     NaN  
              Feke            1      40       3             0     NaN  

[3 rows x 128 columns]

##  3. Birth-Sex
#### 3.1. Birth-Sex ibbs3

In [27]:
file_path = "data/raw/population/birth-sex-ibbs3-2009-2023.xlsx"
dict_sex ={"Cinsiyet:Erkek":"male","Cinsiyet:Kız":"female"}
df_birth_ibbs3 = data_preprocess(file_path, col_dicts=[dict_sex], geo_scale="ibbs3", names= ["sex"])
df_birth_ibbs3.to_csv("data/preprocessed/population/birth-sex-ibbs3-2009-2023.csv")
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_birth_ibbs3.head(2)

male  female
year province               
2009 Adana     19545   18154
     Adıyaman   6808    6607

### 3.2. Birth-Sex-district

In [28]:
file_path = "data/raw/population/birth-sex-district-2014-2023.xlsx"
df_birth_district = data_preprocess(file_path, col_dicts=[dict_sex], geo_scale="district", names=["sex"])
df_birth_district.to_csv("data/preprocessed/population/birth-district-2014-2023.csv")
# after pre-processing, dataframe is saved as csv files. Now let's check the dataframe
df_birth_district.head(2)

male  female
year province district              
2014 Adana    Aladağ     177     160
              Ceyhan    1535    1453

### 4.  Most-Common-Baby-Names-ibbs3

In [16]:
import locale
locale.setlocale(locale.LC_ALL, 'tr_TR.utf8')
from turkish_string import title_tr

In [29]:
df_most_common_baby_names_male = pd.read_excel("data/raw/population/most_common_baby_names_male.xlsx",skiprows=1)
df_most_common_baby_names_male.rename(columns={"Doğum Yılı":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_most_common_baby_names_male[["province","name","sex"]]= df_most_common_baby_names_male[["province","name", "sex"]].map(lambda x: title_tr(x))
df_most_common_baby_names_male.set_index(["year","province"],inplace=True)
df_most_common_baby_names_male.head(2)

name    sex  count  rank
year province                               
2023 Adana     Alparslan  Erkek    223     1
     Adana         Yusuf  Erkek    191     2

#### Merging  Most-Common-Baby-Names with Birth-Sex-ibbs3

In [30]:
df_birth_ibbs3.head(2)

male  female
year province               
2009 Adana     19545   18154
     Adıyaman   6808    6607

In [31]:
df_most_common_baby_names_male = df_most_common_baby_names_male.join(df_birth_ibbs3[["male"]])
df_most_common_baby_names_male.head(2)

name    sex  count  rank   male
year province                                      
2023 Adana     Alparslan  Erkek    223     1  13732
     Adana         Yusuf  Erkek    191     2  13732

                    Save the result to csv file.

In [32]:
df_most_common_baby_names_male.rename(columns={"male":"total_count"},inplace=True)
df_most_common_baby_names_male.to_csv("data/preprocessed/population/most_common_baby_names_male.csv",encoding="utf-8")
df_most_common_baby_names_male.head(2)

name    sex  count  rank  total_count
year province                                            
2023 Adana     Alparslan  Erkek    223     1        13732
     Adana         Yusuf  Erkek    191     2        13732

Let's do the same merging process for female dataset.

In [34]:
df_most_common_baby_names_female = pd.read_excel("data/raw/population/most_common_baby_names_female.xlsx",skiprows=1)
df_most_common_baby_names_female.rename(columns={"Doğum Yılı":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_most_common_baby_names_female[["province","name","sex"]]= df_most_common_baby_names_female[["province","name", "sex"]].map(lambda x: title_tr(x))
df_most_common_baby_names_female.set_index(["year","province"],inplace=True)
df_most_common_baby_names_female.head(2)

name    sex  count  rank
year province                           
2023 Adana      Asel  Kadın    284     1
     Adana     Defne  Kadın    211     2

For female baby names, we repeat the merging(total female births) and saving the resulting dataframe to csv file.

In [35]:
df_most_common_baby_names_female = df_most_common_baby_names_female.join(df_birth_ibbs3[["female"]])
df_most_common_baby_names_female.rename(columns={"female":"total_count"},inplace=True)
df_most_common_baby_names_female.to_csv("data/preprocessed/population/most_common_baby_names_female.csv")
df_most_common_baby_names_female.head(2)

name    sex  count  rank  total_count
year province                                        
2023 Adana      Asel  Kadın    284     1        13236
     Adana     Defne  Kadın    211     2        13236

### 5.  Most Common Names and Surnames-ibbs3

#### 5.1  Most Common Male Names-ibbs3

In [36]:
df_most_common_names_male = pd.read_excel("data/raw/population/most_common_names_male.xlsx",skiprows=1)
df_most_common_names_male.rename(columns={"Yıl":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_most_common_names_male[["province","name","sex"]]= df_most_common_names_male[["province","name", "sex"]].map(lambda x: title_tr(x))
df_most_common_names_male.set_index(["year","province"],inplace=True)
df_most_common_names_male.head(2)

name    sex  count  rank
year province                             
2023 Adana      Mehmet  Erkek  40154     1
     Adana     Mustafa  Erkek  30840     2

In [37]:
df_age_sex = pd.read_csv("data/preprocessed/population/age-sex-ibbs3-2007-2023.csv",index_col=[0,1],header=[0,1])
df_age_sex.head(2)

sex             male                                                           \
age_group        0-4   10-14  15-19  20-24  25-29  30-34  35-39  40-44  45-49   
year province                                                                   
2007 Adana     87875  100851  96648  78022  90764  80670  70980  68068  58558   
     Adıyaman  31114   35241  32872  24563  25786  20867  17573  15887  11527   

sex                   ... female                                            \
age_group      50-54  ...  50-54  55-59    5-9  60-64  65-69  70-74  75-79   
year province         ...                                                    
2007 Adana     52112  ...  51402  39610  93870  27931  22227  16919  13775   
     Adıyaman  10253  ...  10555   9782  32072   7440   6242   4572   3973   

sex                              
age_group     80-84 85-89   90+  
year province                    
2007 Adana     7869  2810  1761  
     Adıyaman  2497  1103   656  

[2 rows x 38 columns]

Merge df_most_common_names_male with "male" column of df_age_sex.

In [38]:
df_most_common_names_male = df_most_common_names_male.join(df_age_sex["male"].sum(axis=1).to_frame("total_count"))
df_most_common_names_male.to_csv("data/preprocessed/population/most_common_names_male.csv")
df_most_common_names_male.head(2)

name    sex  count  rank  total_count
year province                                          
2023 Adana      Mehmet  Erkek  40154     1      1135046
     Adana     Mustafa  Erkek  30840     2      1135046

#### 5.2  Most Common Female Names-ibbs3

Merge df_most_common_names_female with "female" column of df_age_sex

In [39]:
df_most_common_names_female = pd.read_excel("data/raw/population/most_common_names_female.xlsx",skiprows=1,engine='openpyxl')
df_most_common_names_female.rename(columns={"Yıl":"year","İl":"province","İsim":"name","Cinsiyet":"sex","Sayı":"count","Sıra":"rank"}, inplace=True)
df_most_common_names_female[["province","name","sex"]]= df_most_common_names_female[["province","name", "sex"]].map(lambda x: title_tr(x))
df_most_common_names_female.set_index(["year","province"],inplace=True)
df_most_common_names_female.head(2)

name    sex  count  rank
year province                           
2023 Adana     Fatma  Kadın  31705     1
     Adana      Ayşe  Kadın  25029     2

Merge df_most_common_names_female with "female" column of df_age_sex

In [40]:
df_most_common_names_female = df_most_common_names_female.join(df_age_sex["female"].sum(axis=1).to_frame("total_count"))
df_most_common_names_female.to_csv("data/preprocessed/population/most_common_names_female.csv")
df_most_common_names_female.head(2)

name    sex  count  rank  total_count
year province                                        
2023 Adana     Fatma  Kadın  31705     1      1135252
     Adana      Ayşe  Kadın  25029     2      1135252

#### 5.3. Most Common Surnames-ibbs3

In [41]:
df_most_common_surnames = pd.read_excel("data/raw/population/most_common_surnames.xlsx",skiprows=1)
df_most_common_surnames.rename(columns={"Yıl":"year","İl":"province","Soyİsim":"name","Sayı":"count","Sıra":"rank"}, inplace=True)
df_most_common_surnames.drop_duplicates(inplace=True)
df_most_common_surnames[["province","name"]]= df_most_common_surnames[["province","name"]].map(lambda x: title_tr(x))
df_most_common_surnames.set_index(["year","province"],inplace=True)
df_most_common_surnames.head(2)

name  count  rank
year province                     
2023 Adana     Yılmaz  26091     1
     Adana       Kaya  23543     2

In [42]:
df_most_common_surnames = df_most_common_surnames.join(df_age_sex.sum(axis=1).to_frame("total_count"))
df_most_common_surnames.to_csv("data/preprocessed/population/most_common_surnames.csv")
df_most_common_surnames.head(2)

name  count  rank  total_count
year province                                  
2023 Adana     Yılmaz  26091     1      2270298
     Adana       Kaya  23543     2      2270298

## Finally we translate codes to Turkish.

In [6]:
df = pd.read_excel("data/raw/region_codes.xlsx")

# Step 2: Replace 'alt bölgesi' with 'sub-region' in the specific column (assuming the column is named 'Region')
df['sub-region'] = df['sub-region'].str.replace('alt bölgesi', '\nsub-region')

# Step 3: Optionally, save the updated DataFrame back to the Excel file
df.to_excel('data/preprocessed/region_codes.xlsx', index=False)